In [5]:
import sys; sys.path.insert(0, '../')
from plexsim.models import *
import networkx as nx, time
n = 128 
g = nx.grid_graph([n, n])

settings = dict(graph = g,\
                updateType = 'async',\
#                 sampleSize = .25,\
                alpha = 4, #bornholdt
                mu = .2,   # SIRS param
                nu = .02,  # SIRS param
                kappa = .001, # SIRS param
                beta = .2, # SIRS param
                threshold = .01, #CCA param
                p = .1, # Percolation
               t = .5 # ising /potts
               )
ti = time.time()
models = dict(
    Potts = Potts(agentStates = np.arange(4).tolist(),\
          **settings),\
#      Ising = Ising(\
#            **settings),\
#      Bornholdt = Bornholdt(\
#                **settings),\
#      RBN = RBN(**settings),\
#      SIRS = SIRS(\
#          **settings),\
#     CCA = CCA(\
#          agentStates = np.arange(0,4).tolist(),\
#          **settings),\
#     Percolation = Percolation(\
#                 **settings),\
   
)
print(f"Settup time was {time.time() - ti}")
# models.get("Bornholdt").sampleSize = 1
if m := models.get("SIRS"):
    m.states = 0
    m.states[m.sampleNodes(1)[0,0]] = 1
print("starting sims")

T = 1000
import time
start = time.time()
results = np.zeros((len(models), T, g.number_of_nodes()))
for idx, m in enumerate(models.values()):
#     m.states = m.agentStates[-1]
    m.reset()
    results[idx] = m.simulate(T)
r = results.mean(-1).flatten()
results = results.reshape(-1, T, n, n)

# print(results.shape)
# results = results[-1, T,:, :, 0]
print(f'Simulation took {time.time() - start}')

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable as mal
%matplotlib widget
ts = IntSlider(min = 0, max = T - 1, value = 0)

columns, rows = divmod(len(models), 2)
fig, ax = plt.subplots(rows +1, columns +1, figsize = (5,5))
fig.subplots_adjust(wspace = .12, hspace = .0001)
#setup figure/
hs  = []
pad = .01
size = '5%'
for idx, ( (modelName, m), axi) in enumerate(zip(models.items(), ax.flat)):
    div = mal(axi)
    cbr = div.append_axes('bottom', size = size, pad = pad)
#     div = mal(cbr)
#     cbr = div.append_axes('right', size = size, pad = pad)
    AS = m.agentStates
    NS = len(AS) 
    cmap = mpl.colors.LinearSegmentedColormap.from_list(\
          None, plt.cm.viridis(np.linspace(0, 255, NS,\
                                          dtype = int)), NS)
    h = axi.imshow(results[idx, 0], cmap = cmap, vmin = AS[0],\
                  vmax = AS[-1])
    axi.set_title(modelName)
    axi.axis("off")
    tmp = fig.colorbar(h, cax = cbr, ticks = AS,\
                       orientation = 'horizontal')
    if modelName == 'SIRS':
        tmp.set_ticklabels("Sus. Inf. Rec.".split())
    hs.append(h)
mainax = fig.add_subplot(111, frameon = 0,\
                        xticks = [], yticks = [])
[axi.axis('off') for axi in ax.flat[rows * columns:]]
mainax.set_xlabel(f'T = 0')
fig.tight_layout()
@interact(t = ts)
def update(t):
    for idx, h in enumerate(hs):
        h.set_data(results[idx, t])
#         h.autoscale()
    mainax.set_xlabel(f'T = {t}')
    fig.canvas.flush_events()
    fig.canvas.draw()
fig.show()

Settup time was 0.7928941249847412
starting sims
Simulation took 13.702879905700684


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…